In [283]:
import numpy as np

import theano
import theano.tensor as T

In [284]:
# sort of 'hello world' in Theano

x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))
logistic = theano.function([x], s)

out = logistic([[0, 1], [-1, -2]])

## Implementing elasticnet logistic regression (trained with gradient descent) in Theano

In [285]:
from sklearn.exceptions import NotFittedError

n_classes = 10

class LogisticRegression:
    
  def __init__(self, n_iter, wd_coeff, l1_ratio=0, learning_rate=0.001):
    self.n_iter = n_iter
    self.is_fitted = False
    self.l1_ratio = l1_ratio
    self.wd_coeff = wd_coeff
    self.learning_rate = learning_rate 
   
  def fit(self, X, y):
    n_dim = X.shape[1]
    n_classes = len(np.unique(y))
    
    self.thX = T.dmatrix('thX')
    self.thy = T.vector('thy', dtype='int64')
     
    self.thW, self.thB = self.__initialized_weights(n_dim, n_classes)
    
    # calculate probability and loss 
    self.p_y_by_x = T.nnet.softmax(
      T.dot(self.thX, self.thW) + self.thB)
    # negative log likelihood
    nll = (- T.mean(
      T.log(self.p_y_by_x)[T.arange(y_train.shape[0]),self.thy]))
    weight_penalty = T.sum(self.thW ** 2)
    l1_penalty = T.sum(abs(self.thW))
    regularization = (self.wd_coeff * 
                      ((1 - self.l1_ratio) * weight_penalty +
                       self.l1_ratio * l1_penalty)) 
    self.loss = nll + regularization
    
    self.y_pred = T.argmax(self.p_y_by_x, axis=1)
    
    # calculate gradients
    self.gW = T.grad(cost=self.loss, wrt=self.thW)
    self.gB = T.grad(cost=self.loss, wrt=self.thB)
    
    updates = [
      (self.thW,
       self.thW - self.learning_rate * self.gW),
      (self.thB,
         self.thB - self.learning_rate * self.gB)
    ]
    
    # setup training
    self.train_model = theano.function(
      inputs=[self.thX, self.thy],
      outputs=self.loss,
      updates=updates
    )
    
    # actual training
    for __ in range(self.n_iter):
      self.train_model(X, y)
  
    self.is_fitted = True
 
  def predict(self, X):
    if self.is_fitted:
      return self.__prediction_function()(X)
    else:
      raise NotFittedError 
      
       
  def __initialized_weights(self, n_dim, n_classes):
    thW = theano.shared(
        value=np.zeros(
          (n_dim, n_classes),
          dtype=theano.config.floatX),
        name='thW',
        borrow=True)

    # initialize the biases b as a vector of n_out 0s
    thB = theano.shared(
      value=np.zeros(
        (n_classes,),
        dtype=theano.config.floatX),
      name='thB',
      borrow=True)
    return thW, thB
       
  def __prediction_function(self):
    return theano.function(
      inputs=[self.thX],
      outputs=self.y_pred)
  

## Training on MNIST dataset

In [290]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits


digits = load_digits()
X = digits['data']
y = digits['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=0) 

In [287]:
lreg = LogisticRegression(n_iter=100, wd_coeff=0.001, l1_ratio=0.5, learning_rate=0.05)

In [288]:
%time lreg.fit(X_train, y_train)

CPU times: user 584 ms, sys: 8 ms, total: 592 ms
Wall time: 468 ms


In [289]:
from sklearn.metrics import classification_report, accuracy_score 

y_pred = lreg.predict(X_test)
print('accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy: 0.962962962963
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        59
          1       0.88      0.97      0.92        60
          2       1.00      0.98      0.99        59
          3       0.97      0.97      0.97        60
          4       1.00      0.97      0.98        60
          5       0.92      0.95      0.93        60
          6       1.00      0.95      0.97        60
          7       0.94      1.00      0.97        59
          8       0.96      0.88      0.92        58
          9       0.98      0.97      0.97        59

avg / total       0.96      0.96      0.96       594

